![under_construction](figures/under_construction.gif)

I dati utilizzati in questo notebook sono stati presi dalla competizione di Analytics Vidhya [Practice Problem: Big Mart Sales III](https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/#data_dictionary).

# Analisi esplorativa e preprocessamento dei dati

## Indice

In [ ]:
import inspect
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

# 1. Big Mart Sales

## 1.1 Descrizione della competizione

## Problem Statement

The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and find out the sales of each product at a particular store.

Using this model, BigMart will try to understand the properties of products and stores which play a key role in increasing sales.

 

Please note that the data may have missing values as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.

## Data

We have train (8523) and test (5681) data set, train data set has both input and output variable(s). You need to predict the sales for test data set.

|**Variable**                 | Description                                              |
|-----------------------------|----------------------------------------------------------|
|**Item_Identifier**          | Unique product ID                                        |
|**Item_Weight**              | Weight of product                                        |
|**Item_Fat_Content**         | Whether the product is low fat or not                    |
|**Item_Visibility**          | The % of total display area of all products in a store allocated<br/>to the particular product|
|**Item_Type**                |The category to which the product belongs                 |
|**Item_MRP**                 |Maximum Retail Price (list price) of the product          |
|**Outlet_Identifier**        |Unique store ID                                           |
|**Outlet_Establishment_Year**|The year in which store was established                   |
|**Outlet_Size**              |The size of the store in terms of ground area covered     |
|**Outlet_Location_Type**     |The type of city in which the store is located            |
|**Outlet_Type**              |Whether the outlet is just a grocery store or some sort of<br/>supermarket|
|**Item_Outlet_Sales**        |Sales of the product in the particulat store. This is the outcome variable<br/>to be predicted|

### Evaluation Metric

Your model performance will be evaluated on the basis of your prediction of the sales for the test data (test.csv), which contains similar data-points as train except for the sales to be predicted. Your submission needs to be in the format as shown in "SampleSubmission.csv".

We at our end, have the actual sales for the test dataset, against which your predictions will be evaluated. We will use the Root Mean Square Error value to judge your response.

$
RMSE = \sqrt{\frac{\sum_{i=1}^N(Predicted_i - Actual_i)^2}{N}}
$

Where,
$N$: total number of observations
Predicted: the response entered by user
Actual: actual values of sales

Also, note that the test data is further divided into Public (25%) and Private (75%) data. Your initial responses will be checked and scored on the Public data. But, the final rankings will be based on score on Private data set. Since this is a practice problem, we will keep declare winners after specific time intervals and refresh the competition.

## 1.2 Leggere i dati e separare la variabile risposta

### Leggere i dati

In [ ]:
data = pd.read_csv("datasets/big_mart_sales/Train_UWu5bXk.csv")
print("Dimensione del dataset: {} x {}".format(*data.shape))
data.head()

### Dividere le variabili esplicative dalla variabile risposta

In [ ]:
risposta = "Item_Outlet_Sales"
esplicative = sorted(col for col in data.columns if col != risposta)

X, y = data[esplicative].copy(), data[risposta].copy()

# 2. Analisi esplorativa: studiare le variabili esplicative

## 2.1 Dividere le variabili quantitative dalle variabili qualitative

### Controllare i `dtypes` delle colonne

In [ ]:
X.dtypes

### Salvare i nomi delle colonne in due liste distinte

In [ ]:
quantitative = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
qualitative = X.select_dtypes(include=["object"]).columns.tolist()

## 2.2 Variabili quantitative

In [ ]:
X[quantitative].head()

### Contare i valori mancanti

In [ ]:
X[quantitative].isnull().sum()

### Descrivere le variabili

In [ ]:
X.describe() # nota: vengono automaticamente considerate solo le colonne numeriche

## 2.3 Variabili qualitative

In [ ]:
X[qualitative].head()

### Contare i valori mancanti

In [ ]:
X[qualitative].isnull().sum()

### Contare il numero di livelli

In [ ]:
X[qualitative].nunique()

### Contare il numero di osservazioni per ogni livello

In [ ]:
for col in qualitative:
    display(X[col].value_counts().head(16))

### Esercizio

Elencare quanto scoperto grazie all'analisi esplorativa.

### Esercizio

Esplorare le variabili esplicative graficamente (istogrammi, boxplot, ...).

> Suggerimento: considerare le librerie [Matplotlib](https://matplotlib.org/), [Seaborn](https://seaborn.pydata.org/) o, per grafici interattivi, [Bokeh](https://bokeh.pydata.org/en/latest/).

# 3. Preprocessare i dati

## 3.1 Riempire i valori mancanti

### Studio della relazione tra *Item_Identifier* e *Item_Weight*

In [ ]:
weight_grby_id = X[["Item_Identifier", "Item_Weight"]].groupby("Item_Identifier").\
    agg(["count", "min", "max", "mean"])["Item_Weight"]
weight_grby_id.sort_values("count", inplace=True, ascending=False)

print("Item_Identifier senza nemmeno un Item_Weight associato: {}".format((weight_grby_id["count"] == 0).sum()))
weight_grby_id.head()

### Riempire i valori mancanti di *Item_Weight*

In [ ]:
from msbd.preprocessamento import RiempireNAItemWeight

print(inspect.getsource(RiempireNAItemWeight))

In [ ]:
print("Valori mancanti di Item_Weight prima della sostituzione: {}".format(X["Item_Weight"].isnull().sum()))

riempire_na_item_weight = RiempireNAItemWeight()

X = riempire_na_item_weight.fit_transform(X)

print("Valori mancanti di Item_Weight dopo della sostituzione: {}".format(X["Item_Weight"].isnull().sum()))

### Studiare la relazione tra *Outlet_Location_Type* e *Outlet_Size*

In [ ]:
size_grby_location = X.groupby("Outlet_Location_Type")["Outlet_Size"].value_counts().unstack().fillna(0)

size_grby_location

### Studiare la relazione tra *Outlet_Type* e *Outlet_Size*

In [ ]:
size_grby_type = X.groupby("Outlet_Type")["Outlet_Size"].value_counts().unstack().fillna(0)

size_grby_type

### Riempire i valori mancanti di *Outlet_Size*

In [ ]:
from msbd.preprocessamento import RiempireNAOutletSize

print(inspect.getsource(RiempireNAOutletSize))

In [ ]:
print("Valori mancanti di Outlet_Size prima della sostituzione: {}".format(X["Outlet_Size"].isnull().sum()))

riempire_na_outlet_size = RiempireNAOutletSize()

X = riempire_na_outlet_size.fit_transform(X)

print("Valori mancanti di Outlet_Size dopo della sostituzione: {}".format(X["Outlet_Size"].isnull().sum()))

### Riempire gli ultimi valori mancanti rimasti utilizzando la media

In [ ]:
from msbd.preprocessamento import RiempireNAMedia

print(inspect.getsource(RiempireNAMedia))

In [ ]:
print("Valori mancanti prima della sostituzione: \n{}".format(X.isnull().sum()))

riempire_na_media = RiempireNAMedia()

X = riempire_na_media.fit_transform(X)

print("\nValori mancanti dopo della sostituzione: \n{}".format(X.isnull().sum()))

## 3.2 Aggregare i livelli simili delle variabili qualitative

### Aggregare i livelli simili di *Item_Fat_Content*

In [ ]:
from msbd.preprocessamento import Sostituire

print(inspect.getsource(Sostituire))

### Esercizio

Definire il dizionario `to_replace`, da utilizzare per inizializzare l'istanza della classe `Sostituire`, con lo scopo di aggregare i livelli simili di *Item_Fat_Content*.

In [ ]:
to_replace = {"Item_Fat_Content": {"LF": "Low Fat", "low fat": "Low Fat", "reg": "Regular"}}

In [ ]:
print("Livelli prima della sostituzione: {}".format(X["Item_Fat_Content"].unique()))

sostituire_item_fat_content = Sostituire(to_replace)

X = sostituire_item_fat_content.fit_transform(X)

print("Livelli dopo la sostituzione: {}".format(X["Item_Fat_Content"].unique()))

## 3.3 Eliminare le colonne che non si intendono utilizzare

### Eliminare *Item_Identifier*

In [ ]:
from msbd.preprocessamento import Eliminare

print(inspect.getsource(Eliminare))

In [ ]:
elim = Eliminare("Item_Identifier")

X = elim.fit_transform(X)

esplicative.remove("Item_Identifier")
qualitative.remove("Item_Identifier")

# 4. Ottenere e salvare gli indici degli insiemi *training*, *validation* e *test*

## 4.1 Ottenere gli indici degli insiemi di *training*, *validation* e *test*

In [ ]:
from sklearn.model_selection import train_test_split

idx_train, idx_test = train_test_split(y.index.values, test_size=1000)
idx_train, idx_val = train_test_split(idx_train, test_size=1000)

print("Dimensione del training set: {}".format(len(idx_train)))
print("Dimensione del validation set: {}".format(len(idx_val)))
print("Dimensione del test set: {}".format(len(idx_test)))

## 4.2 Salvare gli indici di *training*, *validation* e *test*

In [ ]:
FILE = "datasets/big_mart_sales/{}.npy"

np.save(FILE.format("idx_train"), idx_train)
np.save(FILE.format("idx_val"), idx_val)
np.save(FILE.format("idx_test"), idx_test)

# 5. Analisi esplorativa: studiare la relazione tra variabili esplicative e variabile risposta

<div class="alert alert-danger fade in">
<strong>IMPORTANTE</strong>: Le analisi relative (anche) alla variabile risposta vanno effettuate utilizzando solo l'insieme di <em>training</em>. Utilizzare in questa fase anche gli insiemi di <em>validation</em> e/o di <em>test</em> può inficiare in modo più o meno grave le conclusioni che si traggono su di essi.
</div>

## 5.1 Variabili quantitative

### Esercizio

Studiare la relazione tra variabili quantitative e variabile risposta.

## 5.2 Variabili qualitative

In [ ]:
from msbd.grafici import grafico_barre_qualitative_risposta

print(inspect.getsource(grafico_barre_qualitative_risposta))

In [ ]:
plt.figure(figsize=(10, 10))

grafico_barre_qualitative_risposta(X.loc[idx_train], y.loc[idx_train], qualitative, n_columns=2)

plt.show()

# 6. Trasformare le variabili qualitative in dummy

In [ ]:
from msbd.preprocessamento import OttenereDummy

print(inspect.getsource(OttenereDummy))

### Esercizio

Per come è definito, il metodo `fit()` della classe `OttenereDummy` crea le variabili dummy solo per salvarne i nomi. Ottenere lo stesso risultato senza utilizzare la funzione `get_dummies()` e senza creare le dummy in `fit()` (le dummy verranno create in `transform()`).

In [ ]:
print("X prima della creazione delle variabili dummy:")
display(X.head(2))

od = OttenereDummy(drop_first=True)

X = od.fit_transform(X)

print("\nX dopo la creazione delle variabili dummy:")
X.head(2)

### Esercizio

Perché abbiamo scelto `drop_first=True`?

# 7. Pipeline di preprocessamento

In [ ]:
from sklearn.pipeline import Pipeline

## 7.1 Definire una `Pipeline()`

In [ ]:
preproc = Pipeline([
    ("riempire_na_item_weight", RiempireNAItemWeight()),
    ("riempire_na_outlet_size", RiempireNAOutletSize()),
    ("riempire_na_media", RiempireNAMedia()),
    ("eliminare_item_identifier", Eliminare(columns="Item_Identifier")),
    ("ottenere_dummy", OttenereDummy(drop_first=True)),
])

## 7.2 Trasformare i dati attraverso la pipeline

In [ ]:
print("Dati 'grezzi':")
display(data.head(2))

data_preproc = preproc.fit_transform(data)

print("Dati preprocessati:")
data_preproc.head(2)

### Esercizio

Verificare che la pipeline costruita applichi tutte le trasformazioni viste in questo notebook analizzando `data_preproc`.

## 7.3 Salvare i dati trasformati e la pipeline definita